<img width="8%" alt="Pyvis.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Pyvis.png" style="border-radius: 15%">

# Pyvis - Vizualise ABI network

**Tags:** #pyvis #python #naas #asset #network #analytics

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

## Input

### Import libraries

In [ ]:
from pyvis.network import Network
import naas
from naas_drivers import gsheet
import math
import requests

### Setup variables
**Inputs**
- `abi_spreadsheet`: Google Sheets spreadsheet URL
- `sheet_name`: Sheet name

**Outputs**
- `network_name`: network viz name

In [ ]:
# Inputs
abi_spreadsheet = naas.secret.get("ABI_SPREADSHEET")
sheet_name = "ABI_ONTOLOGY"

# Outputs
network_path = "assets/network.html"

## Model

### Get data from Google Sheet spreadsheet

In [ ]:
df_gsheet = gsheet.connect(abi_spreadsheet).get(sheet_name=sheet_name).fillna("NA")
df_gsheet.head(1)

### Create Pyvis

In [ ]:
def generate_coordinates(radius, num_points, initial_angle):
    coordinates = []
    for i in range(num_points):
        angle = math.radians(initial_angle + i*(360/num_points))
        x = round(radius * math.cos(angle))
        y = round(radius * math.sin(angle))
        coordinates.append((x, y))
    return coordinates

net = Network(
    notebook=True, height="100%", width="100%", bgcolor="#222222", font_color="lightgrey"
)
angle_start = 210

core_coordinates = generate_coordinates(200, 6, angle_start)

tables_coordinates = generate_coordinates(400, 16, angle_start-360/16)

tools_coordinates = generate_coordinates(600, 48, angle_start-360/48)

# Physics solver. Choose one from Pyvis documentation
net.force_atlas_2based()
tables_count = 0
tools_count = 0
add_edges = []
for index, row in df_gsheet.iterrows():
    node = row["Node"]
    node_id = int(row["Node ID"])
    parent = row["Parent"]
    parent_id = int(row["Parent ID"])
    node_type = row["Node Type"]
    
    # Add node
    if node_type == "Main":
        net.add_node(node_id, label=node, color="#47dd82", level=1, fixed=True, x=0, y=0)
    elif node_type == "Core" and node == "📲 Content":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[0][0], y=core_coordinates[0][1], physics=False)
    elif node_type == "Core" and node == "🚀 Growth":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[1][0], y=core_coordinates[1][1], physics=False)
    elif node_type == "Core" and node == "⚡️ Sales":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[2][0], y=core_coordinates[2][1], physics=False)
    elif node_type == "Core" and node == "⚙️ Operations":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[3][0], y=core_coordinates[3][1], physics=False)
    elif node_type == "Core" and node == "💰 Finance":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[4][0], y=core_coordinates[4][1], physics=False)
    elif node_type == "Core" and node == "🌍 Open Data":
        net.add_node(node_id, label=node, color="#ffab76", level=2, fixed=True, x=core_coordinates[5][0], y=core_coordinates[5][1], physics=False)
    elif node_type == "Table":
        net.add_node(node_id, label=node, color="white", level=3, fixed=True, x=tables_coordinates[tables_count][0], y=tables_coordinates[tables_count][1], physics=False)
        tables_count += 1
    elif node_type == "Tool":
        image_url_base = f"https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/{node}"
        image_url = "https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png"
        for x in ["png", "jpeg", "jpg"]:
            res = requests.get(f"{image_url_base}.{x}")
            if res.status_code == 200:
                image_url = f"{image_url_base}.{x}"
                break
        net.add_node(node_id, level=4, fixed=True, x=tools_coordinates[tools_count][0], y=tools_coordinates[tools_count][1], physics=False, shape="circularImage", image=image_url)
        tools_count += 1
        
    if not node_type in ["Main", "Tool"] and parent_id != 0:
        add_edges.append((parent_id, node_id))
        
net.add_edges(add_edges)
network = net.show(network_path)
network

## Outputs

### Create naas asset

In [ ]:
naas.asset.add(network_path, {"inline": True})